In [58]:
import numpy as np
from numpy.core.fromnumeric import shape, size
from random import random, randint, uniform
from keras.datasets import mnist

Stand_Alone functions


In [75]:
def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def rand(row, column):
    returned = np.empty((row, column))
    for j in range(row):
        rando=[]
        for i in range(column):
            rando.append(uniform(-2, 2))
            i+=1
        returned[j] = rando
    return returned


def Normalize(X):
    return((X-X.min())/(X.max()-X.min()))

def Binarize(X):
    X [X < 100] = 0
    X [X >= 100] = 1       
    return X

def labelVector (num):
    n = np.empty((len(num), 10))
    for i in range (len(num)):
        re = np.zeros((1 , 10))
        re[:,num[i]] = 1
        n[i] =re
    return n

def getMax(ma):
    for i in range(shape(ma)[0]):
        index = 0
        for j in range((shape(ma)[1])-1):
            if(ma[i][j+1]>ma[i][index]):
                index = j+1       
        for l in range(shape(ma)[1]):
            if(l==index):
                ma[i][l] =1 
            else:
                ma[i][l] =0      
    return ma


def accuracy(predicted_y, y):
    guess_wrong=0
    for i in range(len(predicted_y)):
        if(not np.array_equiv(predicted_y[i],y[i])):
            guess_wrong+=1
    acc = ((len(predicted_y)- guess_wrong) / len(predicted_y))
    return (acc*100)

Neural Networks Architecture 


In [3]:
class Layer:
    nerweight =0
    inputs = 0
    outputs=0
    delta=0
    def __init__(self, neuronNumber,length):
        self.nerweight =rand(neuronNumber, length)
        #self.bais = rand(neuronNumber , 1)

    def feedForward(self, inputs):
        self.outputs= []
        self.inputs = inputs
        self.outputs  = sigmoid(np.dot(np.array(self.nerweight), self.inputs))
        return self.outputs 

    
    def fit(self, eta, target, outj, sumDelta):
        if(size(sumDelta)==0):
            self.delta=(target-self.outputs)*self.outputs*(1-self.outputs)
            dw = eta*np.transpose([self.delta])*np.transpose(outj)
            self.nerweight = self.nerweight + dw 
            return np.dot(self.delta, self.nerweight)
        else:
            self.delta=sumDelta*self.outputs*(1-self.outputs)
            dw = eta*self.delta*np.transpose([outj])
            self.nerweight = self.nerweight + np.transpose(dw)
            return np.dot(self.delta, self.nerweight)
            

In [38]:
class Network:
    layers = 0
    layernum=0
    length=0
    orin=0
    inputs =0
    target =0
    def __init__(self, NumNeuron, inputs, target):
        self.layers = []
        self.orin = inputs
        self.inputs =inputs
        self.target = target
        length = shape(inputs)[1]
        #One hidden layer layer
        if(len(NumNeuron) ==0):
            layer = Layer(4, length)  
            length =4
            self.layers.append(layer)
        else: 
            for i in range (len(NumNeuron)):
                if(NumNeuron[i]<4):
                    NumNeuron[i] = 4
                layer = Layer(NumNeuron[i], length)  
                length = NumNeuron[i]   
                self.layers.append(layer)
                
        #default Output layer
        layer = Layer(shape(target)[1], length)  
        self.layers.append(layer)

    def feedForward(self, data):
        current =np.copy(data)
        for i in range(len(self.layers)):
            current =self.layers[i].feedForward(current)
        return current
    
    def fit(self, eta, leranRate):
        for j in range(leranRate):
            precdicated = np.empty((shape(self.target))) 
            for k in range((shape(self.orin)[0])): 
                precdicated[k] =self.feedForward(self.orin[k])
                sumdelta = []
                for i in range(len(self.layers)-1, 0, -1):
                    sumdelta =self.layers[i].fit(eta, self.target[k], self.layers[i-1].outputs, sumdelta)
                sumdelta =self.layers[0].fit(eta, self.target[k], self.orin[k], sumdelta)
            if(j%10 == 0):
                print (accuracy(getMax(precdicated), train_y))


Image Processing Functions 


In [5]:
def centroid(X):
    M00=np.sum(X.sum(axis=1))
    M10 =np.array([])
    M01 =np.array([])
    for i in range(X.shape[0]):
        M10= np.append(M10, (X[i].sum(axis=0)*(i+1)))
        
    for i in range(X.shape[1]):
        M01 = np.append(M01,(X[:,i].sum(axis=0)*(i+1)))
    M10 = np.sum(M10)
    M01 = np.sum(M01)
    if(M00==0):
        Xc= 0
        Yc= 0
    else:
        Xc = M10/M00
        Yc = M01/M00
    return np.array([Xc, Yc])

In [6]:
def FeatureImage(X):
    vector=np.array([])
    for i in range(2):
        for j in range(4):              #0-7, 0--14  
            vector= np.append(vector, centroid(X[j*7:((j+1)*7) ,i*14:((i+1)*14)])) 
    return vector

Application

In [7]:
(train_X, train_y), (test_X, test_y) = mnist.load_data()


In [8]:
train_X =Binarize(train_X)
test_X = Binarize(test_X)

In [9]:
vec = np.array([])
for i in range(60000):
    vec = np.append(vec, FeatureImage(train_X[i]))
vec.shape

(960000,)

In [10]:
vec =vec.reshape(60000, 16)
vec.shape

(60000, 16)

In [11]:
vecTest = np.array([])
for i in range(10000):
    vecTest = np.append(vecTest, FeatureImage(test_X[i]))
print(vecTest.shape)
vecTest =vecTest.reshape(10000, 16)
print(vecTest.shape)

(160000,)
(10000, 16)


In [12]:
test_y = labelVector(test_y)


In [13]:
train_y =labelVector(train_y)

TEST

In [14]:
vec=Normalize(vec)

In [42]:
Test4 = Network([6, 12], vec[0:10000], train_y[0:10000])
Test4.fit(0.05, 101)


10.7
39.379999999999995
52.56999999999999
61.919999999999995
66.4
68.30000000000001
69.58
70.5
71.21
71.89
72.72999999999999


In [16]:
vecTest = Normalize(vecTest)

In [17]:
len(test_y)

10000

In [23]:
cle= np.copy(vecTest[0])
cle

array([0.        , 0.        , 0.15546218, 0.75210084, 0.5       ,
       1.        , 0.28571429, 0.91517857, 0.        , 0.        ,
       0.26571429, 0.35428571, 0.28571429, 0.21428571, 0.10714286,
       0.07142857])

In [44]:
preY6 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY6[i]=Test4.feedForward(vecTest[i])
print(accuracy(getMax(preY6), test_y)) #-4 2

70.74000000000001


In [45]:
Test1 = Network([8, 12, 6], vec[0:10000], train_y[0:10000])
Test1.fit(0.03, 200)

12.5
36.66
40.35
43.76
46.68
49.75
54.2
56.68
64.45
66.61
67.75999999999999
68.83
69.56
70.28
70.98
71.49
71.99
72.33000000000001
72.59
72.56


In [46]:
preY1 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY1[i]=Test1.feedForward(vecTest[i])
print(accuracy(getMax(preY1), test_y))

71.50999999999999


In [48]:
Test2 = Network([8, 5, 4], vec[0:10000], train_y[0:10000])
Test2.fit(0.05, 200)
preY2 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY2[i]=Test2.feedForward(vecTest[i])
print(accuracy(getMax(preY2), test_y))

11.04
19.939999999999998
27.47
33.650000000000006
34.03
34.47
38.96
47.56
52.349999999999994
53.37
54.87
57.02
58.5
60.419999999999995
62.370000000000005
64.03999999999999
65.64
66.35
66.99000000000001
67.14
66.55


In [49]:
Test3 = Network([8, 5, 4], vec[0:30000], train_y[0:30000])
Test3.fit(0.05, 200)
preY3 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY3[i]=Test3.feedForward(vecTest[i])
print(accuracy(getMax(preY3), test_y)) #-4 4

14.646666666666667
34.17333333333333
36.77333333333333
37.7
44.053333333333335
46.54333333333333
51.57000000000001
53.833333333333336
55.17999999999999
55.943333333333335
56.45666666666666
56.86333333333333
57.9
61.21666666666666
62.14666666666666
63.06
64.75666666666666
65.61
66.23333333333333
66.90666666666667
69.22


In [53]:
Test100 = Network([6, 5, 8], vec[0:30000], train_y[0:30000])
Test100.fit(0.01, 100)
preY100 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY100[i]=Test100.feedForward(vecTest[i])
print(accuracy(getMax(preY100), test_y)) 

9.916666666666666
14.716666666666667
28.746666666666666
43.156666666666666
48.75
55.04333333333334
56.81666666666667
58.42333333333334
59.11
61.18666666666667
63.81


In [55]:
Test10 = Network([6, 5, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.01, 100)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-4, 4

10.37
28.736666666666665
42.39
54.32
60.23
63.05
64.71000000000001
66.18
67.11666666666667
67.84333333333333
69.46


In [56]:
Test10 = Network([6, 5, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.03, 100)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2 ,2

12.593333333333334
55.39333333333334
63.00666666666667
66.38666666666667
67.88666666666666
69.51666666666667
70.03
70.47666666666666
70.87333333333333
71.26333333333334
71.89


In [57]:
Test10 = Network([6, 15, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.05, 100)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 integer

16.513333333333332
56.50333333333334
64.47333333333334
72.31666666666666
74.93666666666667
75.91
76.79
77.03666666666666
77.34333333333333
77.67666666666668
77.08


In [59]:
Test10 = Network([6, 15, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.05, 100)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

21.833333333333332
68.28
74.52333333333333
77.46666666666667
78.48666666666666
78.96333333333332
79.4
79.67999999999999
79.78333333333333
79.92666666666666
79.75999999999999


In [60]:
Test10 = Network([6, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.04, 100)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

19.843333333333334
58.13333333333334
68.69666666666666
71.89
73.78333333333333
74.95666666666668
75.53666666666666
75.98666666666666
76.22666666666666
76.44999999999999
76.75999999999999


In [63]:
Test10 = Network([6, 4, 8], vec[0:60000], train_y[0:60000])
Test10.fit(0.04, 200)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

29.39333333333333
66.07499999999999
69.695
71.72666666666667
72.93666666666667
73.66666666666667
75.07000000000001
75.47666666666667
75.88333333333334
76.26333333333334
76.40166666666667
76.71666666666667
76.96
76.98
77.00666666666666
77.11833333333334
77.19500000000001
77.24166666666666
77.285
77.285
76.91


In [64]:
Test10 = Network([6, 14, 8], vec[0:60000], train_y[0:60000])
Test10.fit(0.05, 101)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

20.866666666666667
69.31333333333333
74.46166666666667
76.46166666666666
77.46
78.095
78.44166666666666
78.72333333333333
78.92333333333333
79.08333333333334
79.145
78.42


In [65]:
Test10 = Network([6, 14, 8], vec[0:60000], train_y[0:60000])
Test10.fit(0.01, 101)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

17.391666666666666
47.58
59.67166666666667
65.25999999999999
68.73
72.47666666666667
73.94333333333334
75.305
76.69833333333334
77.35
77.78166666666667
78.14


In [66]:
Test10 = Network([12], vec[0:60000], train_y[0:60000])
Test10.fit(0.05, 101)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

33.22
76.25999999999999
78.99333333333334
80.47166666666666
81.44500000000001
82.16166666666666
82.55166666666666
82.88666666666667
83.17
83.41666666666666
84.09
84.66


In [67]:
Test10 = Network([10, 14, 8], vec[0:60000], train_y[0:60000])
Test10.fit(0.05, 201)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

25.08666666666667
77.81666666666666
81.62166666666667
83.11166666666666
83.89333333333333
84.88833333333334
85.34666666666666
85.55666666666667
85.79833333333333
85.90333333333334
86.03333333333333
86.125
86.265
86.48666666666666
86.58500000000001
86.62166666666667
86.73666666666666
86.83333333333333
86.97166666666666
87.03333333333333
87.11166666666666
86.77


In [74]:
Test10 = Network([6, 15, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.05, 101)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-3, 3 uniform

19.066666666666666
69.60333333333332
76.37333333333333
77.75333333333333
78.32666666666667
78.80666666666667
78.93666666666667
79.11666666666667
79.21666666666667
79.36666666666666
79.59333333333333
79.14


In [ ]:
Test10 = Network([6, 15, 8], vec[0:30000], train_y[0:30000])
Test10.fit(0.05, 200)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #- uniform #run when I'm awake

In [76]:
Test20= Network([], vec[0:30000], train_y[0:30000])
Test20.fit(0.05, 200)
preY20 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY20[i]=Test20.feedForward(vecTest[i])
print(accuracy(getMax(preY20), test_y))

15.893333333333334
48.156666666666666
59.56
60.21333333333333
60.35666666666667
60.42999999999999
60.473333333333336
60.51333333333333
60.56666666666667
60.596666666666664
60.68666666666667
60.66666666666667
60.74666666666667
60.68666666666667
60.633333333333326
60.64000000000001
60.6
60.60333333333333
60.633333333333326
60.653333333333336
60.589999999999996


In [77]:
Test20= Network([20, 18], vec[0:30000], train_y[0:30000])
Test20.fit(0.05, 201)
preY20 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY20[i]=Test20.feedForward(vecTest[i])
print(accuracy(getMax(preY20), test_y))

30.793333333333333
80.05
82.84
84.33666666666667
85.39666666666666
86.63
87.37
88.07000000000001
88.57666666666667
88.86666666666667
89.05999999999999
89.33
89.50666666666666
89.64999999999999
89.80333333333334
89.86333333333333
89.96
90.10666666666667
90.21333333333334
90.29666666666667
90.45666666666666
89.72


In [79]:
Test10 = Network([35, 20], vec[0:60000], train_y[0:60000])
Test10.fit(0.05, 301)
preY10 = np.empty((len(test_y), 10))
for i in range((len(test_y))):
    preY10[i]=Test10.feedForward(vecTest[i])
print(accuracy(getMax(preY10), test_y)) #-2, 2 uniform

52.739999999999995
84.27166666666666
86.62666666666667
87.94833333333332
88.42999999999999
88.76666666666667
88.96166666666666
89.16
89.34166666666667
89.60666666666667
91.10833333333333
91.39333333333333
91.50166666666667
91.61500000000001
91.69666666666667
91.80666666666667
91.89
91.96833333333333
92.02666666666667
92.07833333333333
92.12666666666667
92.19833333333334
92.27
92.36833333333333
92.44833333333334
92.525
92.56666666666666
92.625
92.64833333333333
92.67666666666666
92.70833333333334
91.97
